In [22]:
import pandas as pd
from haversine import haversine, Unit
from obspy.taup import TauPyModel
from src.utils.data_reading.sound_data.sound_file_manager import DatFilesManager
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np

In [25]:
FILE = 'C:/Users/Romain/PycharmProjects/toolbox/data/recensement_stations_OHASISBIO_RS.csv'
stations = pd.read_csv(FILE )
stations = stations.loc[stations.year==2018]
stations
station_dic = {}
for name in stations.station_name :
    st_lat, st_lon  = stations.loc[stations.station_name==name].lat.values[0],\
    stations.loc[stations.station_name==name].lon.values[0]
    station_dic[name] = [st_lat, st_lon]
    print(name , st_lat, st_lon)

ELAN -56.4602 62.976
MADE -24.2053 63.0102
MADW -29.0473 54.258
NEAMS -31.5758 83.2423
RTJ -24.3792 72.372
SSEIR -33.5175 70.8657
SSWIR -38.5465 52.9287
SWAMS-bot -42.9513 74.5975
WKER2 -46.6015 60.5475


In [35]:
station_dic['ELAN']

[-56.4602, 62.976]

In [36]:
# Define station coordinates and initialize model

name = 'ELAN'
model = TauPyModel(model="ak135")

[st_lat, st_lon]  =  station_dic[name]
def calculate_phases(row):
    """
    Compute travel times and metadata for all phases in `phase_list`.
    Returns a list of dictionaries (one per phase).
    """

    event_lat = row['latitude']
    event_lon = row['longitude']
    event_depth = row['depth']  # Ensure depth is in kilometers
    origin_time = row['time']

    # Calculate angular distance between event and station
    dist_deg = haversine(
        (event_lat, event_lon), (st_lat, st_lon), unit=Unit.DEGREES
    )

    # Get travel times for all phases in the list
    try:
        arrivals = model.get_travel_times(
            source_depth_in_km=event_depth,
            distance_in_degree=dist_deg,
            phase_list=["P", "PKP", "PKiKP", "PKIKP"]
        )
    except ValueError:
        return []  # Return empty list if no arrivals

    phases = []
    for arrival in arrivals:
        phase_info = {
            'phase': arrival.name,
            'travel_time': arrival.time,
            'arrival_time': origin_time + pd.to_timedelta(arrival.time, unit='s'),
            'distance_deg': dist_deg
        }
        phases.append(phase_info)

    return phases

# Load catalogue and compute phases
PATH = 'C:/Users/Romain/PycharmProjects/NEIC_ISC_join/data'
NAME_NEIC = '/NEIC_2018_M6.csv'
catalogue = pd.read_csv(PATH + NAME_NEIC, parse_dates=['time'], date_format='ISO8601')

# Expand DataFrame to include all phases
catalogue['phases'] = catalogue.apply(calculate_phases, axis=1)
catalogue = catalogue.explode('phases').reset_index(drop=True)

# Extract phase details into columns
phase_data = pd.json_normalize(catalogue['phases'])
catalogue = pd.concat([catalogue.drop('phases', axis=1), phase_data], axis=1)

# Drop rows where phases were not found (optional)
catalogue = catalogue.dropna(subset=['phase'])

# Save the extended catalogue
# catalogue.to_csv(PATH + '/NEIC_2018_M6_with_all_phases.csv', index=False)

In [37]:
catalogue.sort_values("time", inplace=True, ignore_index=True)


In [38]:
dat = ['2018-02-25 17:44:44.140000','2018-03-25 20:14:47.690000','2018-03-26 09:51:00.430000','2018-04-02 13:40:34.840000','2018-07-28 22:47:38.740000','2018-08-05 11:46:38.630000','2018-08-19 00:19:40.670000','2018-08-19 14:56:27.490000','2018-09-06 15:49:18.710000','2018-09-10 04:19:02.630000','2018-09-28 10:02:45.250000','2018-10-29 06:54:21.250000','2018-12-05 04:18:08.420000','2018-12-11 02:26:29.420000','2018-12-29 03:39:09.740000']
Err =  [2.66,2.21,2.39,2.01,2.32 ,2.14,1.61,1.78,2.16,1.93,2.03,1.60,1.5,1.88,2.94]
# for i, l in enumerate(dat) :
#     print(catalogue.loc[catalogue['time']==l]['rms'].unique()*2.0)

catalogue.loc[catalogue['time']==dat[6]]

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,depthError,magError,magNst,status,locationSource,magSource,phase,travel_time,arrival_time,distance_deg
134,2018-08-19 00:19:40.670000+00:00,-18.1125,-178.153,600.0,8.2,mww,NaN,13.0,3.63,0.79,...,1.9,0.045,47.0,reviewed,us,us,P,715.080061,2018-08-19 00:31:35.750061216+00:00,89.681135
135,2018-08-19 00:19:40.670000+00:00,-18.1125,-178.153,600.0,8.2,mww,NaN,13.0,3.63,0.79,...,1.9,0.045,47.0,reviewed,us,us,PKiKP,1007.658253,2018-08-19 00:36:28.328253192+00:00,89.681135


In [ ]:
# Function to downsample audio data
def downsample_audio(data, original_fs, target_fs):
    """Downsample audio data to the target frequency"""
    # print(f"Downsampling from {original_fs}Hz to {target_fs}Hz")
    # Calculate downsampling factor
    factor = int(original_fs / target_fs)
    # Apply anti-aliasing filter before downsampling
    b, a = signal.butter(5, target_fs/2, fs=original_fs, btype='low')
    filtered_data = signal.filtfilt(b, a, data)
    # Downsample by taking every 'factor' sample
    downsampled_data = filtered_data[::factor]
    return downsampled_data, target_fs

# Function to apply dehazing (using spectral subtraction)
def dehaze_audio(data, fs, frame_size=1024, overlap=0.8):
    """Apply spectral subtraction for dehazing"""
    # print("Applying dehazing using spectral subtraction")
    hop_size = int(frame_size * (1 - overlap))
    # Estimate noise profile from first few frames
    num_noise_frames = 5
    noise_estimate = np.zeros(frame_size // 2 + 1)

    frames = []
    for i in range(0, len(data) - frame_size, hop_size):
        frame = data[i:i+frame_size]
        if len(frame) < frame_size:
            frame = np.pad(frame, (0, frame_size - len(frame)))
        frames.append(frame)

    # Estimate noise from first few frames
    for i in range(min(num_noise_frames, len(frames))):
        noise_frame = frames[i]
        noise_spectrum = np.abs(np.fft.rfft(noise_frame * np.hanning(frame_size)))
        noise_estimate += noise_spectrum / num_noise_frames

    # Apply spectral subtraction
    result = np.zeros(len(data))
    window = np.hanning(frame_size)

    for i, frame in enumerate(frames):
        windowed_frame = frame * window
        spectrum = np.fft.rfft(windowed_frame)
        magnitude = np.abs(spectrum)
        phase = np.angle(spectrum)

        # Subtract noise and ensure no negative values
        magnitude = np.maximum(magnitude - noise_estimate * 1.5, 0.01 * magnitude)

        # Reconstruct frame
        enhanced_spectrum = magnitude * np.exp(1j * phase)
        enhanced_frame = np.fft.irfft(enhanced_spectrum)

        # Overlap-add
        start = i * hop_size
        end = start + frame_size
        result[start:end] += enhanced_frame

    # Normalize
    result = result / np.max(np.abs(result))
    return result

# Function to apply Butterworth bandpass filter
def apply_butter_bandpass(data, fs, lowcut, highcut, order=5):
    """Apply Butterworth bandpass filter"""
    # print(f"Applying bandpass filter: {lowcut}-{highcut}Hz, order {order}")
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

# Function to create spectrogram
def create_spectrogram(data, fs, nperseg=256, noverlap=128, cmap='viridis'):
    """Create and return spectrogram of the data"""
    # print("Creating spectrogram")
    f, t, Sxx = signal.spectrogram(data, fs=fs, nperseg=nperseg, noverlap=noverlap)
    return f, t, Sxx

# Function to detect potential seismic events using energy
def detect_seismic_events(data, fs, window_size=5.0, threshold_factor=5.0):
    """Detect potential seismic events based on energy threshold"""
    print("Detecting potential seismic events")
    window_samples = int(window_size * fs)
    energy = []

    # Calculate energy in sliding windows
    for i in range(0, len(data) - window_samples, window_samples // 2):
        window = data[i:i+window_samples]
        window_energy = np.sum(window**2) / len(window)
        energy.append(window_energy)

    # Set threshold as a factor of the median energy
    energy = np.array(energy)
    threshold = np.median(energy) * threshold_factor

    # Find events that exceed threshold
    events = []
    in_event = False
    event_start = 0

    for i, e in enumerate(energy):
        if e > threshold and not in_event:
            in_event = True
            event_start = i * (window_samples // 2) / fs
        elif e <= threshold and in_event:
            in_event = False
            event_end = i * (window_samples // 2) / fs
            events.append((event_start, event_end))

    # Handle if we're still in an event at the end
    if in_event:
        event_end = len(data) / fs
        events.append((event_start, event_end))

    return events, energy, threshold

# Main processing function
def process_underwater_recording(data,df,date_time,original_fs=240., target_fs=50, low_pass=1.5, high_pass=0.6):
    """Process underwater recording to visualize seismic events"""
    # Load data

    print(f"Original sampling rate: {original_fs}Hz")
    print(f"Original data length: {len(data)} samples ({len(data)/original_fs:.2f} seconds)")

    # Downsample to 50Hz
    downsampled_data, new_fs = downsample_audio(data, original_fs, target_fs)
    print(f"Downsampled data length: {len(downsampled_data)} samples ({len(downsampled_data)/new_fs:.2f} seconds)")

    # Apply dehazing
    dehazed_data = dehaze_audio(downsampled_data, new_fs)

    # Apply Butterworth bandpass filter
    filtered_data = apply_butter_bandpass(dehazed_data, new_fs, high_pass, low_pass)

    # Detect potential seismic events
    events, energy, threshold = detect_seismic_events(filtered_data, new_fs)
    time = timedelta(minutes=10)
    for event_start, _ in events:
        if np.abs(time - event_start) < 10 :
            catalogue.loc[catalogue['time'] == date_time,"candidate"] = True
            break
    if True :
        # Create a separate figure for event detection results
        plt.figure(figsize=(10, 4))
        window_size = 5.0  # seconds
        for arrival, phase in zip(df['arrival_time'],df['phase']):
            print(arrival, phase)
            time = arrival - df['arrival_time'].iloc[0] + timedelta(minutes=10)
            time = time.total_seconds()
            plt.axvline(time, color='g', linestyle='--', label=phase)
        time_axis = np.arange(len(energy)) * (window_size/2)
        plt.plot(time_axis, energy)
        plt.axhline(threshold, color='r', linestyle='--', label='Threshold')
        plt.title('Signal Energy for Event Detection')
        plt.xlabel('Time (s)')
        plt.ylabel('Energy')
        plt.legend()
        plt.show()
    return filtered_data, events

In [ ]:
# Configuration
from tqdm.notebook import tqdm
PATH = f"F:/OHASISBIO/2018/{name}"
ORIGINAL_FS = 240.0
TARGET_FS = 60
LOW_PASS = 1.5
HIGH_PASS = 0.6
TIME_TOLERANCE = 10  # seconds

# Initialize candidate column
catalogue['candidate'] = False
catalogue['file_number'] = -1
manager = DatFilesManager(PATH)

def find_candidates(manager, catalogue):
    ORIGINAL_FS = 240.0
    TARGET_FS = 60
    LOW_PASS = 1.5
    HIGH_PASS = 0.6
    TIME_TOLERANCE = 10
    # Process each unique event in the catalogue
    for date_time in tqdm(catalogue['time'].unique()):

        event_df = catalogue[catalogue['time'] == date_time]
        first_arrival = event_df['arrival_time'].min()

        # Define time window: 10 minutes before and after first arrival
        start = (first_arrival - timedelta(minutes=10)).replace(tzinfo=None)
        end = (first_arrival + timedelta(minutes=10)).replace(tzinfo=None)

        try:
            # Load and preprocess seismic data
            data = manager.get_segment(start, end)
            file_number = manager.find_file_name(start)
            downsampled_data, new_fs = downsample_audio(data, ORIGINAL_FS, TARGET_FS)
            dehazed_data = dehaze_audio(downsampled_data, new_fs)
            filtered_data = apply_butter_bandpass(dehazed_data, new_fs, HIGH_PASS, LOW_PASS)

            # Detect seismic events (adjust threshold as needed)
            events, energy, threshold = detect_seismic_events(filtered_data, new_fs, threshold_factor=8.0)

            # # Check if any detected event aligns with expected arrival time
            # expected_time_sec = 600  # 10 minutes into the segment
            # for event_start, _ in events:
            #     if abs(event_start - expected_time_sec) < TIME_TOLERANCE:
            #         catalogue.loc[catalogue['time'] == datetime, 'candidate'] = True
            #         break  # Stop checking once a match is found
            event_starts = np.array([e[0] for e in events])
            if np.any(np.abs(event_starts - 600) < TIME_TOLERANCE):
                catalogue.loc[catalogue['time'] == date_time, 'candidate'] = True
                catalogue.loc[catalogue['time'] == date_time, 'file_number'] = file_number

            # Optional: Plot detection results for debugging
            if False:  # Set to True to enable
                plt.figure(figsize=(10, 4))
                time_axis = np.arange(len(energy)) * (5.0 / 2)  # Assuming 5s window
                plt.plot(time_axis, energy)
                plt.axhline(threshold, color='r', linestyle='--', label='Threshold')
                plt.axvline(expected_time_sec, color='g', linestyle='--', label='Expected Arrival')
                plt.xlabel('Time (s)')
                plt.ylabel('Energy')
                plt.legend()
                plt.show()

        except Exception as e:
            print(f"Error processing {date_time}: {str(e)}")
            continue  # Skip to next event on failure
    return data, catalogue

data, catalogue = find_candidates(manager, catalogue)

#first arrival is in the middle
#second arrival plot will be
#dt = sec - first + timedelta(minute=10) and ect.



In [ ]:
# data.shape

In [ ]:
# plt.figure()
# time = np.arange(len(data))/240.
# plt.plot(time, data)
# df = catalogue[catalogue['time'] == datetime]
# for arrivals in df['arrival_time']:
#     time = arrivals - df['arrival_time'].iloc[0] + timedelta(minutes=10)
#     time = time.total_seconds()
#     plt.axvline(time, color='r', linestyle='--', label='Arrivals')

In [ ]:
catalogue[catalogue["candidate"]==True]['time'].unique()

In [ ]:
catalogue.to_csv(f'../../../../data/{name}_2018_M6.csv',index=False)

In [ ]:
catalogue[catalogue['candidate'] == True]['file_number'].unique()

# *ISC ERROR Scrapping*


In [34]:
import requests
import xml.etree.ElementTree as ET
import csv
from datetime import datetime

# List of event timestamps
timestamps = [
    "2018-02-25 17:44:44.140000",
    "2018-03-25 20:14:47.690000",
    "2018-03-26 09:51:00.430000",
    "2018-04-02 13:40:34.840000",
    "2018-07-28 22:47:38.740000",
    "2018-08-05 11:46:38.630000",
    "2018-08-19 00:19:40.670000",
    "2018-08-19 14:56:27.490000",
    "2018-09-06 15:49:18.710000",
    "2018-09-10 04:19:02.630000",
    "2018-09-28 10:02:45.250000",
    "2018-10-29 06:54:21.250000",
    "2018-12-05 04:18:08.420000",
    "2018-12-11 02:26:29.420000",
    "2018-12-29 03:39:09.740000",
]

# Base URL for ISC QuakeML queries
BASE_URL = "https://www.isc.ac.uk/cgi-bin/web-db-run"

# Function to query ISC and get uncertainty
def get_uncertainty(event_time):
    # Convert event time to ISC format
    dt = datetime.strptime(event_time, "%Y-%m-%d %H:%M:%S.%f")
    search_time = dt.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-4]  # Keep only the first two digits of milliseconds

    # Construct API request URL
    params = {
        "request": "COMPREHENSIVE",
        "out_format": "QuakeML",
        "start_year": dt.year,
        "start_month": dt.month,
        "start_day": dt.day,
        "start_time": (dt- timedelta(seconds=60)).strftime("%H:%M:%S"),
        "end_year": dt.year,
        "end_month": dt.month,
        "end_day": dt.day,
        "end_time": (dt+ timedelta(seconds=60)).strftime("%H:%M:%S"),
    }

    response = requests.get(BASE_URL, params=params)

    # Check if the response is successful
    if response.status_code != 200:
        print(f"Request failed for {event_time} with status code {response.status_code}")
        return None  # Return None if request failed

    # Check if the response body is empty
    if not response.text.strip():
        print(f"Empty response for {event_time}")
        return None  # Return None if no content in the response

    try:
        # Parse XML response
        root = ET.fromstring(response.text)
    except ET.ParseError as e:
        print(f"XML parsing error for {event_time}: {e}")
        return None  # Return None if XML parsing fails

    # Find the exact matching event
    for origin in root.findall(".//{*}origin"):
        time_elem = origin.find("{*}time/{*}value")
        uncertainty_elem = origin.find("{*}time/{*}uncertainty")
        depth_elem = origin.find("{*}depth/{*}value")

        if time_elem is not None and uncertainty_elem is not None:
            event_time_utc = time_elem.text
            event_time_utc = event_time_utc.replace("Z", "")  # Remove 'Z' for direct comparison

            # Match the timestamp to milliseconds (only first two digits of milliseconds)
            event_time_utc_ms = event_time_utc[:23]  # First two digits of milliseconds
            search_time_ms = search_time[:23]  # First two digits of milliseconds

            # Debug print: show the event time comparison for debugging
            print(f"Checking: Requested Time: {search_time_ms} vs Event Time: {event_time_utc_ms}")

            # Compare timestamps with only first two digits of milliseconds
            if event_time_utc_ms == search_time_ms:  # Compare only up to two digits of milliseconds
                uncertainty = uncertainty_elem.text
                depth = depth_elem.text
                return event_time_utc, uncertainty, depth

    return None

# Store results
results = []

# Loop through each timestamp and get uncertainty
for timestamp in timestamps:
    data = get_uncertainty(timestamp)
    if data:
        event_time, uncertainty, depth = data
        depth = float(depth)*1e-3 #converting to Km
        results.append([timestamp, event_time, uncertainty, depth])
        print(f"Event: {timestamp} -> Uncertainty: {uncertainty} | Depth: {depth} km")
    else:
        results.append([timestamp, "Not Found", "N/A"])
        print(f"Event: {timestamp} -> No data found")

# Save results to CSV
csv_filename = "seismic_uncertainty_results.csv"
with open(csv_filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Requested Time", "Event Time (UTC)", "Uncertainty"])
    writer.writerows(results)

print(f"\nResults saved to {csv_filename}")


Checking: Requested Time: 2018-02-25T17:44:44.14 vs Event Time: 2018-02-25T17:44:44.32
Checking: Requested Time: 2018-02-25T17:44:44.14 vs Event Time: 2018-02-25T17:44:39.65
Checking: Requested Time: 2018-02-25T17:44:44.14 vs Event Time: 2018-02-25T17:44:40.60
Checking: Requested Time: 2018-02-25T17:44:44.14 vs Event Time: 2018-02-25T17:44:43.82
Checking: Requested Time: 2018-02-25T17:44:44.14 vs Event Time: 2018-02-25T17:44:44.14
Event: 2018-02-25 17:44:44.140000 -> Uncertainty: 2.66 | Depth: 25.21 km
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event Time: 2018-03-25T20:14:15.72
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event Time: 2018-03-25T20:14:15.23
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event Time: 2018-03-25T20:14:16.08
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event Time: 2018-03-25T20:14:28.31
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event Time: 2018-03-25T20:14:32.50
Checking: Requested Time: 2018-03-25T20:14:47.69 vs Event